In [67]:
import pandas as pd
import sqlalchemy, os
import requests
from binance import BinanceSocketManager
from binance.client import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta

load_dotenv()

API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")

client = Client(API_KEY, API_SECRET)
# client = Client()

In [7]:
import sqlite3

db = sqlite3.connect('crypto.db')

cursor = db.cursor()

try:
    cursor.execute("""
        CREATE TABLE test (
            symbol TEXT,
            avg_price REAL,
            qty INTEGER,
            buy_time REAL
        )""")
except:
    print("Table test already exists")

try:
    cursor.execute("""
        CREATE TABLE test2 (
            symbol TEXT,
            unblocktime REAL
        )""")
except:
    print("Table test2 already exists")

db.commit()
db.close()

In [6]:
# Function for printing to files
def print_file(ls, file):
    with open(str(file)+'.txt','w') as file:
        file.write(str(ls)[1:-1].replace(' ','').replace('\'',''))
        
# Function for getting price
def get_price(ls):
    avg = (float(ls[1]) + float(ls[4])) / 2
    return avg

# Function for getting percent change in 6 hour
def get_percent_change(ls_1, ls_2):
    change = (get_price(ls_2) - get_price(ls_1)) / get_price(ls_1) * 100
    return float("{:.4f}".format(change))

# Function for flitering symbols using threshold
def filter_symbol(level, list_of_symbols, threshold, time):
    now_time = datetime.now()
    prev_time = now_time - time
    selected_symbols = []

    for symbol in list_of_symbols:
        klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1MINUTE, str(prev_time), str(now_time))

        percent_change = get_percent_change(klines[0], klines[-1])

        if percent_change > threshold:
            selected_symbols.append(symbol)
            if(level==4):
                print(f"{symbol}: {percent_change}% ${get_price(klines[0])}")
    print_file(selected_symbols,'pairs_level_'+str(level))
    if(level==4) & (len(selected_symbols)>0):
        print_file(selected_symbols,'selected_symbols')
        print('---------------------------\n')
    return selected_symbols

In [73]:
def fetchall(table='test'):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(f"SELECT * FROM {table}")
    alldata = cursor.fetchall()
    db.commit()
    db.close()
    return alldata

def extract_from_tuple(alldata, i = 0):
    symbols = [x[i] for x in alldata]
    return symbols

def execute(command):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(command)
    db.commit()
    db.close()
    
def add_to_db(symbol, price, qty):
    time = int(datetime.utcnow().strftime('%d%H%M%S'))
    execute(f"INSERT INTO test VALUES ('{symbol}',{price},{qty},{time})")
    print(f"Bought {qty} units of {symbol} at ${price:.2f}")

def delete_from_db(symbol,price,qty,profit):
    # execute(f"DELETE FROM test WHERE symbol = '{symbol}'")
    print(f"Sold {qty} units of {symbol} at ${price:.2f}. Profit {profit:.2f}%")
    
def get_data(symbol,table='test'):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(f"SELECT * FROM {table} WHERE symbol = '{symbol}'")
    data = cursor.fetchone()
    db.commit()
    db.close()
    return data

def extract_from_tuple(alldata, i = 0):
    symbols = [x[i] for x in alldata]
    return symbols

def block(symbol):
    time = int((datetime.utcnow() + timedelta(hours=1)).strftime('%d%H%M%S'))
    execute(f"INSERT INTO block VALUES ('{symbol}',{time})")

In [ ]:
# date_1 = datetime.utcnow().strftime('%y %m %d %H:%M:%S.%f')
# date_2 = datetime.utcnow().strftime('%d%H%M%S%f')
date_1 = datetime.utcnow().strftime('%y %m %d %H:%M:%S')
date_2 = datetime.utcnow().strftime('%d%H%M%S')
print(date_1)
print(int(date_2))

In [9]:
#First get ETH price
eth_price = client.get_symbol_ticker(symbol="ETHUSDT")

# Calculate how much ETH $200 can buy
buy_quantity = round(200 / float(eth_price['price']))

# Create test order
order = client.create_test_order(
        symbol='ETHUSDT',
        side=Client.SIDE_BUY,
        type=Client.ORDER_TYPE_MARKET,
        quantity=buy_quantity
    )

 # The 200 in buy_quantity is the amount of money you want to spend on ETH.


13


In [61]:
price = client.get_symbol_ticker(symbol="BCCUSDT")['price']
print(price)
price = float(client.get_avg_price(symbol="ETHUSDT")['price'])
print(price)

448.70000000
1196.29800688


In [7]:
info = client.get_exchange_info()['symbols']
for symbol in info:
    if 'DOGEUSDT' in symbol['symbol']:
        for j in symbol:
            print(j,symbol[j])
        print('\n')
        for item in symbol['filters']:
            print(item)
# pricePrecision = info['symbols'][0]['pricePrecision']
# print(pricePrecision)
print("------------------------------------------------------------------------------------------------")
ticker = client.get_symbol_info("DOGEUSDT")['filters'][2]
print(ticker)

symbol DOGEUSDT
status TRADING
baseAsset DOGE
baseAssetPrecision 8
quoteAsset USDT
quotePrecision 8
quoteAssetPrecision 8
baseCommissionPrecision 8
quoteCommissionPrecision 8
orderTypes ['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_LIMIT', 'TAKE_PROFIT_LIMIT']
icebergAllowed True
ocoAllowed True
quoteOrderQtyMarketAllowed True
allowTrailingStop True
cancelReplaceAllowed False
isSpotTradingAllowed True
isMarginTradingAllowed True
filters [{'filterType': 'PRICE_FILTER', 'minPrice': '0.00001000', 'maxPrice': '1000.00000000', 'tickSize': '0.00001000'}, {'filterType': 'PERCENT_PRICE', 'multiplierUp': '5', 'multiplierDown': '0.2', 'avgPriceMins': 5}, {'filterType': 'LOT_SIZE', 'minQty': '1.00000000', 'maxQty': '9000000.00000000', 'stepSize': '1.00000000'}, {'filterType': 'MIN_NOTIONAL', 'minNotional': '10.00000000', 'applyToMarket': True, 'avgPriceMins': 5}, {'filterType': 'ICEBERG_PARTS', 'limit': 10}, {'filterType': 'MARKET_LOT_SIZE', 'minQty': '0.00000000', 'maxQty': '16552468.66597222', 

In [19]:
order = client.create_order(symbol="CHRUSDT", side="SELL", type="MARKET", quantity=57)
print(order)

{'symbol': 'CHRUSDT', 'orderId': 728550331, 'orderListId': -1, 'clientOrderId': 'hupXWtikPkFksvnXZE770i', 'transactTime': 1655715199569, 'price': '0.00000000', 'origQty': '57.00000000', 'executedQty': '57.00000000', 'cummulativeQuoteQty': '10.07190000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'fills': [{'price': '0.17670000', 'qty': '57.00000000', 'commission': '0.00003588', 'commissionAsset': 'BNB', 'tradeId': 65667100}]}


In [97]:
import time
real_time(time.time())
print((datetime.now()))

1970-01-20 10:02:59
2022-06-25 23:48:58.906508


In [93]:
def real_time(epoch):
    epoch = int(epoch/1000)
    print(datetime.fromtimestamp(epoch).strftime('%Y-%m-%d %H:%M:%S'))